# Phase 1 Manual Testing Notebook
This notebook provides hands-on, cell-by-cell checks for the wardrobe taxonomy, data model, SQLite-backed store, and ADK wardrobe tools introduced in Phase 1.

## Setup and Imports
Import the core Phase 1 modules. If you run this notebook outside the repo root, make sure the project directory is on `PYTHONPATH` so imports resolve correctly.

In [ ]:
# Ensure the repository root is on PYTHONPATH for clean imports
import sys
from pathlib import Path

repo_root = Path('..').resolve()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

# Core Phase 1 imports
from models.taxonomy import (
    CATEGORIES,
    SEASON_TAGS,
    STYLE_TAGS,
    normalize_color_name,
    validate_category,
    validate_subcategory,
)
from models.wardrobe_item import WardrobeItem, from_raw_metadata
from tools.wardrobe_store import SQLiteWardrobeStore
from tools.wardrobe_tools import WardrobeTools
from adk_app.app import FashionConciergeApp
from adk_app.config import ADKConfig


## Initialize a Temporary Test Database
Use a throwaway SQLite file so manual experiments do not touch real data.

In [ ]:
# Create a temporary SQLite database file for safe experimentation
import tempfile

temp_db = tempfile.NamedTemporaryFile(suffix='.db', delete=False)
TEMP_DB_PATH = Path(temp_db.name)
temp_db.close()

store = SQLiteWardrobeStore(TEMP_DB_PATH)
print(f'Using temporary database at: {TEMP_DB_PATH}')


## Test Taxonomy
Explore canonical categories, validation helpers, and color normalization.

In [ ]:
# List canonical categories and their subcategories
CATEGORIES


In [ ]:
# Validate a correct category
valid_category = validate_category('Top')
print('Validated category:', valid_category)


In [ ]:
# Demonstrate an invalid category raising an error
try:
    validate_category('gadget')
except ValueError as exc:
    print('Expected error:', exc)


In [ ]:
# Normalize a few common color variants
colors = ['navy blue', 'Sky Blue', 'Off White', 'charcoal']
normalised = {c: normalize_color_name(c) for c in colors}
normalised


## Test WardrobeItem Creation
Create valid and invalid items to observe normalization and validation behavior.

In [ ]:
# Build a valid WardrobeItem directly
from dataclasses import asdict

valid_item = WardrobeItem(
    item_id='item_001',
    user_id='user_demo',
    image_url='https://example.com/image1.jpg',
    source_url='https://example.com/product1',
    category='top',
    sub_category='shirt',
    colors=['Navy Blue', 'White'],
    materials=['cotton', 'linen'],
    brand='DemoBrand',
    fit='slim',
    season_tags=['warm_weather'],
    style_tags=['business', 'casual'],
    user_notes='Sample item for manual testing.',
)
asdict(valid_item)


In [ ]:
# Build a WardrobeItem via the ingestion-friendly factory
raw_metadata = {
    'item_id': 'item_002',
    'user_id': 'user_demo',
    'image_url': 'https://example.com/image2.jpg',
    'source_url': 'https://example.com/product2',
    'category': 'bottom',
    'sub_category': 'jeans',
    'colors': ['black'],
    'materials': ['denim'],
    'season_tags': ['all_year'],
    'style_tags': ['street'],
}
factory_item = from_raw_metadata(raw_metadata)
asdict(factory_item)


In [ ]:
# Invalid combinations raise clear errors (e.g., wrong subcategory)
try:
    WardrobeItem(
        item_id='bad_item',
        user_id='user_demo',
        image_url='https://example.com/bad.jpg',
        source_url='https://example.com/bad',
        category='shoes',
        sub_category='jeans',  # invalid on purpose
    )
except ValueError as exc:
    print('Expected validation error:', exc)


## Test WardrobeStore Basic Operations
Use the temporary store to exercise CRUD behavior scoped to a user.

In [ ]:
# Create and store an item for test_user
test_user = 'test_user'
base_item = WardrobeItem(
    item_id='store_item_1',
    user_id=test_user,
    image_url='https://example.com/store1.jpg',
    source_url='https://example.com/store1',
    category='outerwear',
    sub_category='coat',
    colors=['beige'],
    materials=['wool'],
    season_tags=['cold_weather'],
    style_tags=['formal'],
)
store.create_item(base_item)
fetched = store.get_item(test_user, 'store_item_1')
print('Fetched item:', fetched)


In [ ]:
# Add multiple items and list them for the user
additional = [
    WardrobeItem(
        item_id='store_item_2',
        user_id=test_user,
        image_url='https://example.com/store2.jpg',
        source_url='https://example.com/store2',
        category='top',
        sub_category='hoodie',
        colors=['gray'],
        materials=['cotton'],
        season_tags=['cold_weather'],
        style_tags=['casual'],
    ),
    WardrobeItem(
        item_id='store_item_3',
        user_id=test_user,
        image_url='https://example.com/store3.jpg',
        source_url='https://example.com/store3',
        category='shoes',
        sub_category='sneakers',
        colors=['white'],
        materials=['leather'],
        season_tags=['all_year'],
        style_tags=['street'],
    ),
]
for item in additional:
    store.create_item(item)

all_items = store.list_items_for_user(test_user)
print('Items for test_user:', all_items)


In [ ]:
# Update an item and refetch it
updated = store.update_item(test_user, 'store_item_2', {'brand': 'ComfyCo', 'colors': ['charcoal']})
print('Updated item:', updated)


In [ ]:
# Delete an item and confirm removal
store.delete_item(test_user, 'store_item_3')
print('After delete, get_item returns:', store.get_item(test_user, 'store_item_3'))


## Test Search Functionality
Insert a small, diverse set of items and exercise category, season, color, and style filters.

In [ ]:
# Create items for a dedicated search user
search_user = 'search_user'
search_items = [
    WardrobeItem(
        item_id='search_top_casual',
        user_id=search_user,
        image_url='https://example.com/search1.jpg',
        source_url='https://example.com/search1',
        category='top',
        sub_category='tee',
        colors=['blue'],
        materials=['cotton'],
        season_tags=['warm_weather'],
        style_tags=['casual'],
    ),
    WardrobeItem(
        item_id='search_bottom_business',
        user_id=search_user,
        image_url='https://example.com/search2.jpg',
        source_url='https://example.com/search2',
        category='bottom',
        sub_category='chinos',
        colors=['beige'],
        materials=['cotton'],
        season_tags=['all_year'],
        style_tags=['business'],
    ),
    WardrobeItem(
        item_id='search_outer_cold',
        user_id=search_user,
        image_url='https://example.com/search3.jpg',
        source_url='https://example.com/search3',
        category='outerwear',
        sub_category='puffer',
        colors=['black'],
        materials=['synthetic'],
        season_tags=['cold_weather'],
        style_tags=['street'],
    ),
]
for item in search_items:
    store.create_item(item)
print('Ready with search corpus for user:', search_user)


In [ ]:
# Search by category
category_results = store.search_items(search_user, {'category': 'top'})
print('Category=top results:', category_results)


In [ ]:
# Search by season tags
season_results = store.search_items(search_user, {'season_tags': ['cold_weather']})
print('Season cold_weather results:', season_results)


In [ ]:
# Search by colors
color_results = store.search_items(search_user, {'colors': ['navy', 'black']})
print('Color navy/black results:', color_results)


In [ ]:
# Search by style tags
style_results = store.search_items(search_user, {'style_tags': ['business']})
print('Style business results:', style_results)


## Test Wardrobe Tools (ADK function wrappers)
Instantiate the FashionConciergeApp in lightweight mode and call wardrobe tools directly to compare with store behavior.

In [ ]:
# Configure a lightweight app instance pointing at the temporary DB
import os

# Provide a local API key placeholder so genai.configure does not fail
os.environ.setdefault('GOOGLE_API_KEY', 'local-demo-key')

app_config = ADKConfig(
    project_id='fashion-concierge-local',
    location='us-central1',
    api_key=os.environ.get('GOOGLE_API_KEY'),
    wardrobe_db_path=str(TEMP_DB_PATH),
)
app = FashionConciergeApp(config=app_config)

tool_wrapper = app.wardrobe_tools
print('Wardrobe tools registered:', [t.name for t in app.wardrobe_tool_defs])


In [ ]:
# Use the tool wrappers to add and list items
tool_user = 'tool_user'
tool_item = tool_wrapper.add_wardrobe_item(
    tool_user,
    {
        'item_id': 'tool_item_1',
        'image_url': 'https://example.com/tool_item.jpg',
        'source_url': 'https://example.com/tool_source',
        'category': 'dress',
        'sub_category': 'day_dress',
        'colors': ['red'],
        'materials': ['cotton'],
        'season_tags': ['warm_weather'],
        'style_tags': ['party'],
    },
)
print('Created via tool:', tool_item)
print('List via tool:', tool_wrapper.list_wardrobe_items(tool_user))


## End-to-End Mini Flow
Walk through creation, storage, retrieval, and search for a single item to mirror a typical user flow.

In [ ]:
# Create, insert, retrieve, and search for an item
flow_user = 'flow_user'
flow_item_data = {
    'item_id': 'flow_item',
    'image_url': 'https://example.com/flow.jpg',
    'source_url': 'https://example.com/flow',
    'category': 'shoes',
    'sub_category': 'boots',
    'colors': ['brown'],
    'materials': ['leather'],
    'season_tags': ['cold_weather'],
    'style_tags': ['casual'],
}

print('#1 Construct item via factory')
flow_item = from_raw_metadata({**flow_item_data, 'user_id': flow_user})
print(flow_item)

print('
#2 Store item')
store.create_item(flow_item)

print('
#3 Retrieve stored item')
print(store.get_item(flow_user, 'flow_item'))

print('
#4 Search for the item by category and season')
results = store.search_items(flow_user, {'category': 'shoes', 'season_tags': ['cold_weather']})
print(results)
